In [1]:
%load_ext autoreload
%autoreload 2
from data.input_data import get_all_data
from pyomo.environ import *
from model.sets_params import define_sets_and_params
from model.variables import define_variables
from model.objective import define_objective
# from model.solver_GLPK import solve_model
# from model.solver_CBC import solve_model
from model.solver_scip import solve_model
# from model.solver_scip_1 import solve_model
from model.constraints_copy import add_constraints
# 1. Chargement des données
data = get_all_data()
# 2. Création du modèle Pyomo
model = ConcreteModel()
model = define_sets_and_params(model, data)
define_variables(model)
model = add_constraints(model, data)  # ou juste add_constraints(model, data)
define_objective(model)
# 3. Affichage des informations du modèle
num_vars = sum(1 for v in model.component_objects(Var, active=True) for _ in v)
num_constraints = sum(1 for c in model.component_objects(Constraint, active=True) for _ in c)

print(f"Nombre total de variables     : {num_vars}")
print(f"Nombre total de contraintes  : {num_constraints}")

# Variables binaires
num_bin_vars = sum(1 for v in model.component_objects(Var, active=True)
                for index in v if v[index].domain == Binary)
print(f"Nombre de variables binaires : {num_bin_vars}")

# Variables continues
num_cont_vars = sum(1 for v in model.component_objects(Var, active=True)
                    for index in v if v[index].domain == NonNegativeReals)
print(f"Nombre de variables continues: {num_cont_vars}")
# 3. Résolution
result = solve_model(model, tee=True)
model.write('modele.lp', io_options={'symbolic_solver_labels': True})
    
print("\n------ QUALITÉ DES COMPOSANTS (ratio) ---------")       
for c in model.C:
    for k in list(model.K)[:2]:
        mu = model.mu_ck[c, k].value or 0
        D = model.D_k[k]
        ratio = mu / D if D > 0 else 0
        j = model.lamda_k[k]
        bmin = model.BetaMin_cj[j, c]
        bmax = model.BetaMax_cj[j, c]
        print(f"c={c}, k={k}: ratio={ratio:.4f}  [{bmin}, {bmax}]")

Saving matrix Distor_ihc with dimensions 20x5x4
Nombre total de variables     : 5145
Nombre total de contraintes  : 10861
Nombre de variables binaires : 2436
Nombre de variables continues: 12
✅ SCIP disponible. Lancement de la résolution avec gap = 0.9...
SCIP version 9.0.1 [precision: 8 byte] [memory: block] [mode: optimized] [LP solver: Soplex 7.0.1] [GitHash: bebb64304e]
Copyright (c) 2002-2024 Zuse Institute Berlin (ZIB)

External libraries: 
  Soplex 7.0.1         Linear Programming Solver developed at Zuse Institute Berlin (soplex.zib.de) [GitHash: 1cc71921]
  CppAD 20180000.0     Algorithmic Differentiation of C++ algorithms developed by B. Bell (github.com/coin-or/CppAD)
  MPIR 3.0.0           Multiple Precision Integers and Rationals Library developed by W. Hart (mpir.org)
  ZIMPL 3.6.0          Zuse Institute Mathematical Programming Language developed by T. Koch (zimpl.zib.de)
  AMPL/MP 690e9e7      AMPL .nl file reader library (github.com/ampl/mp)
  PaPILO 2.2.1         par

In [2]:
import pandas as pd

def extract_blending_results(model):
    # Récupération des noms (labels)
    i_names = model.I_names
    h_names = model.H_names
    k_names = model.K_names
    j_names = model.J_names
    stock_sources = model.Stock_sources
    E_k = model.E_k
    L_k = model.L_k

    # Pour retrouver la qualité demandée pour chaque commande k
    # lamda_k est l'indice du produit QM associé à la commande k
    get_qm = lambda k: j_names[model.lamda_k[k]]

    results = []
    for (i, h, k, t) in model.x_ihkt:
        val = model.x_ihkt[i, h, k, t].value
        binary_val = model.o_ihkt[i, h, k, t].value
        if val is not None and val > 1e-3:
            results.append({
                "Commande": k_names[k],
                "Qualité demandée (QM)": get_qm(k),
                "Nom ingrédient": i_names[i],
                "Stock source": stock_sources[i],
                "Gamme": h_names[h],
                "t": t + 1,
                "x_ihkt (tonnes)": val,
                "o_ihkt (binaire)": binary_val,
                "E_k": E_k[k],
                "L_k": L_k[k]
            })
    df = pd.DataFrame(results)
    # Ajout du % blending
    df['Total_commande'] = df.groupby('Commande')['x_ihkt (tonnes)'].transform('sum')
    df['%_blending'] = (df['x_ihkt (tonnes)'] / df['Total_commande']) * 100
    return df

def save_blending_results_to_csv(df, filename='blending_results_table15.csv'):
    df.to_csv(filename, index=False)
    print(f"✅ Résultats enrichis enregistrés dans {filename}")

def export_tableau_15_par_commande(df, export_excel=False):
    import pandas as pd

    if export_excel:
        with pd.ExcelWriter('resultats_tableau_15_par_commande.xlsx') as writer:
            for cmd in df['Commande'].unique():
                df_cmd = df[df['Commande'] == cmd][[
                    'Qualité demandée (QM)', 'Nom ingrédient', 'Stock source', 'Gamme', 't', 'x_ihkt (tonnes)', '%_blending'
                ]].sort_values('x_ihkt (tonnes)', ascending=False)
                df_cmd.to_excel(writer, sheet_name=f"{cmd}", index=False)
        print("✅ Exporté dans 'resultats_tableau_15_par_commande.xlsx' (une feuille par commande)")

    # Affichage console (markdown-friendly)
    for cmd in df['Commande'].unique():
        print(f"\n### 🟢 Commande : **{cmd}** | QM demandée : **{df[df['Commande']==cmd]['Qualité demandée (QM)'].iloc[0]}**\n")
        display_df = df[df['Commande'] == cmd][[
            'Nom ingrédient', 'Stock source', 'Gamme', 't', 'x_ihkt (tonnes)', '%_blending'
        ]].sort_values('x_ihkt (tonnes)', ascending=False)
        print(display_df.to_markdown(index=False, floatfmt=".2f"))
        print(f"\nTotal livré : {display_df['x_ihkt (tonnes)'].sum():,.2f} t")

df = extract_blending_results(model)
save_blending_results_to_csv(df)
df.head()

✅ Résultats enrichis enregistrés dans blending_results_table15.csv


,Commande,Qualité demandée (QM),Nom ingrédient,Stock source,Gamme,t,x_ihkt (tonnes),o_ihkt (binaire),E_k,L_k,Total_commande,%_blending
0,Commande 1,Profil_BG,BT_Sc1,Stock criblé 1,Laverie,2,100.0,1.0,1,4,4020.0,2.487562
1,Commande 1,Profil_BG,PBG_Sc1,Stock criblé 1,sans traitement,2,2837.0,1.0,1,4,4020.0,70.572139
2,Commande 1,Profil_BG,BT_Sc1,Stock criblé 1,sans traitement,3,681.0,1.0,1,4,4020.0,16.940299
3,Commande 1,Profil_BG,TBT_Sc1,Stock criblé 1,sans traitement,4,102.0,1.0,1,4,4020.0,2.537313
4,Commande 1,Profil_BG,PBG_Sc1,Stock criblé 1,sans traitement,4,100.0,1.0,1,4,4020.0,2.487562


In [3]:
df.head(15)  # Affichage des premières lignes du DataFrame

,Commande,Qualité demandée (QM),Nom ingrédient,Stock source,Gamme,t,x_ihkt (tonnes),o_ihkt (binaire),E_k,L_k,Total_commande,%_blending
0,Commande 1,Profil_BG,BT_Sc1,Stock criblé 1,Laverie,2,100.0,1.0,1,4,4020.0,2.487562
1,Commande 1,Profil_BG,PBG_Sc1,Stock criblé 1,sans traitement,2,2837.0,1.0,1,4,4020.0,70.572139
2,Commande 1,Profil_BG,BT_Sc1,Stock criblé 1,sans traitement,3,681.0,1.0,1,4,4020.0,16.940299
3,Commande 1,Profil_BG,TBT_Sc1,Stock criblé 1,sans traitement,4,102.0,1.0,1,4,4020.0,2.537313
4,Commande 1,Profil_BG,PBG_Sc1,Stock criblé 1,sans traitement,4,100.0,1.0,1,4,4020.0,2.487562
5,Commande 1,Profil_BG,MT_Sc1,Stock criblé 1,sans traitement,4,100.0,1.0,1,4,4020.0,2.487562
6,Commande 1,Profil_BG,TBT_Smine,La mine,Laverie,5,100.0,1.0,1,4,4020.0,2.487562
7,Commande 2,Export,MT_Sc1,Stock criblé 1,sans traitement,3,1403.0,1.0,3,6,5160.0,27.189922
8,Commande 2,Export,PBG_Sc1,Stock criblé 1,Laverie,4,146.0,1.0,3,6,5160.0,2.829457
9,Commande 2,Export,MT_Smine,La mine,traitement TM,5,109.0,1.0,3,6,5160.0,2.112403


In [ ]:
import pandas as pd
import dataframe_image as dfi

def export_fusion_style_table(df, filename="blending_table.png"):
    # Colonnes à garder (dans l'ordre)
    cols = [
        'Commande', 'Qualité demandée (QM)', 'Nom ingrédient',
        'Stock source', 'Gamme', 't',
        'x_ihkt (tonnes)', 'Total_commande', '%_blending'
    ]
    df = df[cols].copy()

    # Trier par numéro de commande (extraire le n° si présent)
    df['Commande_num'] = df['Commande'].str.extract(r'(\d+)').astype(float)
    df = df.sort_values(by=['Commande_num', 't'])
    df = df.drop(columns='Commande_num')

    # Masquer les doublons dans la colonne 'Commande'
    df['Commande'] = df['Commande'].where(~df['Commande'].duplicated(), "")

    # Format des colonnes numériques
    df['x_ihkt (tonnes)'] = df['x_ihkt (tonnes)'].round(2)
    df['Total_commande'] = df['Total_commande'].round(2)
    df['%_blending'] = df['%_blending'].map(lambda x: f"{x:.2f}%")

    # Création du style avec table sans index
    styled = (
        df.style
        .hide(axis="index")  # ✅ Supprimer la colonne des index (0,1,2,…)
        .set_table_styles([
            {"selector": "thead th", "props": [("font-weight", "bold"), ("background-color", "#f0f0f0"), ("border", "1px solid black")]},
            {"selector": "td", "props": [("text-align", "center"), ("border", "1px solid black"), ("padding", "6px")]}
        ])
    )

    # Exporter en image (en mode matplotlib pour compatibilité Jupyter)
    dfi.export(styled, filename, table_conversion="matplotlib")
    print(f"✅ Tableau exporté dans : {filename}")
export_fusion_style_table(df, filename="S_blending_table.png")

✅ Tableau exporté dans : S_blending_table.png


In [ ]:
df.head(10)  # Affiche les premières lignes du DataFrame pour vérification

,Commande,Qualité demandée (QM),Nom ingrédient,Stock source,Gamme,t,x_ihkt (tonnes),o_ihkt (binaire),E_k,L_k,Total_commande,%_blending
0,Commande 1,Profil_BG,TBT_Sc1,Stock criblé 1,sans traitement,2,657.0,1.0,1,3,2000.0,32.850000
1,Commande 1,Profil_BG,MT_Sc1,Stock criblé 1,sans traitement,2,1243.0,1.0,1,3,2000.0,62.150000
2,Commande 1,Profil_BG,MT_Sf1,Stock lavé,sans traitement,3,100.0,1.0,1,3,2000.0,5.000000
3,Commande 2,Export,MT_Sf1,Stock lavé,sans traitement,2,1894.0,1.0,1,5,2000.0,94.700000
4,Commande 2,Export,MT_Sc1,Stock criblé 1,sans traitement,3,106.0,1.0,1,5,2000.0,5.300000
5,Commande 3,Tess,PBG_Se,Stock épierré,sans traitement,2,100.0,1.0,2,6,2000.0,5.000000
6,Commande 3,Tess,PBG_Sf1,Stock lavé,sans traitement,3,911.0,1.0,2,6,2000.0,45.550000
7,Commande 3,Tess,PBG_Sf1,Stock lavé,sans traitement,6,989.0,1.0,2,6,2000.0,49.450000
8,Commande 4,BG_BT,PBG_Sc1,Stock criblé 1,sans traitement,3,1530.0,1.0,3,7,2010.0,76.119403
9,Commande 4,BG_BT,PBG_Smine,La mine,traitement TM,5,100.0,1.0,3,7,2010.0,4.975124


In [ ]:
import pandas as pd

def extract_final_stocks(model):
    i_names = model.I_names
    stock_sources = model.Stock_sources
    T_max = max(model.T)  # Dernière période

    results = []
    for i in model.I:
        val = model.S_it[i, T_max].value
        results.append({
            "Nom ingrédient": i_names[i],
            "Stock source": stock_sources[i],
            f"Stock final à t={T_max+1} (tonnes)": val
        })
    df = pd.DataFrame(results)
    return df

def save_final_stocks_to_csv(df, filename='stock_final.csv'):
    df.to_csv(filename, index=False)
    print(f"✅ Stock final enregistré dans {filename}")

def show_final_stocks(df):
    print("\n=== Tableau des stocks finaux ===")
    print(df)

# Utilisation typique (dans main.py ou un notebook)
df_stock = extract_final_stocks(model)
save_final_stocks_to_csv(df_stock)
show_final_stocks(df_stock)


✅ Stock final enregistré dans stock_final.csv

=== Tableau des stocks finaux ===
   Nom ingrédient        Stock source  Stock final à t=29 (tonnes)
0       TBT_Smine             La mine                    1222250.0
1        BT_Smine             La mine                      89000.0
2       PBG_Smine             La mine                     104990.0
3        MT_Smine             La mine                      60600.0
4         TBT_Sc1      Stock criblé 1                          8.0
5          BT_Sc1      Stock criblé 1                          0.0
6         PBG_Sc1      Stock criblé 1                        203.0
7          MT_Sc1      Stock criblé 1                          0.0
8         TBT_Sc2  Stock prés laverie                      13300.0
9          BT_Sc2  Stock prés laverie                      10000.0
10        PBG_Sc2  Stock prés laverie                       6500.0
11         MT_Sc2  Stock prés laverie                        800.0
12         TBT_Se       Stock épierré           

In [ ]:
# def diagnostic_flux_mine(model):
#     print("===== DIAGNOSTIC FLUX MINE =====")
#     for i in model.QS_mines:
#         total = 0
#         for h in model.H:
#             for k in model.K:
#                 for t in model.T:
#                     if (i, h, k, t) in model.IHKT_valid:
#                         val = model.x_ihkt[i, h, k, t].value
#                         if val is not None:
#                             total += val
#         print(f"Ingrédient {i} (Mine): Total extrait sur tout l’horizon = {total}")
#         print(f"Stock initial = {model.Stock_initial_i[i]}")
# diagnostic_flux_mine(model)

In [ ]:
# 1. Demande totale
demande_totale = sum([model.D_k[k] for k in model.K])
print(f"Demande totale sur tout l'horizon (somme D_k) : {demande_totale:.0f} tonnes")

# 2. Stock initial total
stock_initial_total = sum([model.Stock_initial_i[i] for i in model.I])
print(f"Stock initial total (somme Stock_initial_i) : {stock_initial_total:.0f} tonnes")

# 3. Affiche le ratio
if demande_totale > stock_initial_total:
    print(f"\n⚠️  La demande ({demande_totale:.0f}) est supérieure au stock initial ({stock_initial_total:.0f}) !")
else:
    print(f"\nLa demande ({demande_totale:.0f}) est couverte par le stock initial ({stock_initial_total:.0f})")


Demande totale sur tout l'horizon (somme D_k) : 68800 tonnes
Stock initial total (somme Stock_initial_i) : 1242800 tonnes

La demande (68800) est couverte par le stock initial (1242800)


In [ ]:
# Somme des stocks initiaux pour QS_Sc1
stock_QS_Sc1 = sum([model.Stock_initial_i[i] for i in model.QS_Sc1])
print(f"Stock initial total dans QS_Sc1 : {stock_QS_Sc1:.0f} tonnes")

# Somme des stocks initiaux pour QS_Se
stock_QS_Se = sum([model.Stock_initial_i[i] for i in model.QS_Se])
print(f"Stock initial total dans QS_Se  : {stock_QS_Se:.0f} tonnes")

# Somme des stocks initiaux pour QSL_Sf1
stock_QSL_Sf1 = sum([model.Stock_initial_i[i] for i in model.QSL_Sf1])
print(f"Stock initial total dans QSL_Sf1 : {stock_QSL_Sf1:.0f} tonnes")

# Optionnel : total cumulé sur les trois
total_stocks = stock_QS_Sc1 + stock_QS_Se + stock_QSL_Sf1
print(f"Somme totale des stocks initiaux utilisés (QS_Sc1 + QS_Se + QSL_Sf1) : {total_stocks:.0f} tonnes")

# 1. Demande totale
demande_totale = sum([model.D_k[k] for k in model.K])
print(f"Demande totale sur tout l'horizon (somme D_k) : {demande_totale:.0f} tonnes")
# 3. Affiche le ratio
if demande_totale > stock_initial_total:
    print(f"\n⚠️  La demande ({demande_totale:.0f}) est supérieure au stock initial ({total_stocks:.0f}) !")
else:
    print(f"\nLa demande ({demande_totale:.0f}) est couverte par le stock initial ({total_stocks:.0f})")



Stock initial total dans QS_Sc1 : 31000 tonnes
Stock initial total dans QS_Se  : 20300 tonnes
Stock initial total dans QSL_Sf1 : 16000 tonnes
Somme totale des stocks initiaux utilisés (QS_Sc1 + QS_Se + QSL_Sf1) : 67300 tonnes
Demande totale sur tout l'horizon (somme D_k) : 68800 tonnes

La demande (68800) est couverte par le stock initial (67300)


In [ ]:
print("===== CONTRÔLE DES FLUX D’UTILISATION =====")
for i in model.I:
    initial = model.Stock_initial_i[i]
    extrait = sum(model.x_ihkt[i, h, k, t].value for h in model.H for k in model.K for t in model.T if (i, h, k, t) in model.IHKT_valid)
    final = model.S_it[i, model.T.last()].value
    print(f"Ingrédient {i:2} | Stock initial: {initial:8.0f} | Extrait total: {extrait:8.0f} | Stock final: {final:8.0f} | Flux total = {initial + sum(model.Ait[t, i] for t in model.T) - final:8.0f}")


===== CONTRÔLE DES FLUX D’UTILISATION =====
Ingrédient  1 | Stock initial:  1113900 | Extrait total:     3650 | Stock final:  1222250 | Flux total =     3650
Ingrédient  2 | Stock initial:     5000 | Extrait total:        0 | Stock final:    89000 | Flux total =        0
Ingrédient  3 | Stock initial:    21400 | Extrait total:      410 | Stock final:   104990 | Flux total =      410
Ingrédient  4 | Stock initial:     4600 | Extrait total:        0 | Stock final:    60600 | Flux total =        0
Ingrédient  5 | Stock initial:     3000 | Extrait total:     2992 | Stock final:        8 | Flux total =     2992
Ingrédient  6 | Stock initial:     5000 | Extrait total:     5000 | Stock final:        0 | Flux total =     5000
Ingrédient  7 | Stock initial:    21400 | Extrait total:    21197 | Stock final:      203 | Flux total =    21197
Ingrédient  8 | Stock initial:     1600 | Extrait total:     1600 | Stock final:        0 | Flux total =     1600
Ingrédient  9 | Stock initial:    13300 | Ex

In [ ]:
# %load_ext autoreload
# %autoreload 2
# from data.input_data import get_all_data
# from pyomo.environ import *
# from model.sets_params import define_sets_and_params
# from model.variables import define_variables
# from model.objective import define_objective
# # from model.solver_GLPK import solve_model
# # from model.solver_CBC import solve_model
# from model.solver_scip import solve_model
# from model.constraints_copy import add_constraints
# # 1. Chargement des données
# data = get_all_data()
# # 2. Création du modèle Pyomo
# model = ConcreteModel()
# model = define_sets_and_params(model, data)
# define_variables(model)
# model = add_constraints(model, data)  # ou juste add_constraints(model, data)
# define_objective(model)

In [ ]:
# model.test=Param(model.T, initialize=0)  # Exemple d'initialisation